This notebook covers:
- SparkSession fundamentals  
- SparkContext  
- Spark engine concepts  
- RDD creation  
- Transformations vs actions  
- Lazy evaluation  
- Narrow vs wide dependencies  
- Hands-on examples  
📌 This notebook is part of my "PySpark Series: Beginner to Expert".

Medium blog: 

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("TestApp").getOrCreate()

In [ ]:
spark

## ✔ What each line means:
**from pyspark.sql import SparkSession**

Imports the main entry point for PySpark DataFrame API.

**SparkSession.builder**

Starts the configuration builder.
Think of it as filling a form before Spark starts.

**.appName("TestApp")**

Names your application.

Why name?

Shows up in Spark UI

Helps monitor jobs

Helps in cluster logs

**.getOrCreate()**

This is important.

If Spark is not running → create it

If SparkSession already exists → return existing one

This avoids multiple Spark sessions (which can crash JVM).

# ✔ Increase memory

.config("spark.executor.memory", "4g")

.config("spark.driver.memory", "2g")


# ✔ Enable Arrow for faster Pandas conversions

.config("spark.sql.execution.arrow.enabled", "true")

# ✔ Complete example:

In [ ]:
spark = (
    SparkSession.builder
    .appName("MySparkApp")
    .master("local[*]")
    .config("spark.executor.memory", "4g")
    .config("spark.sql.execution.arrow.enabled", "true")
    .getOrCreate()
)

In [ ]:
spark

# 🔥 What Is SparkContext?

* SparkContext = the “engine operator”.

* Manages the connection to the cluster

* Transfers tasks to executors

* Keeps track of job status

* Handles scheduling

You can access it through:

In [ ]:
spark.sparkContext

In [ ]:
sc = spark.sparkContext
print(sc.appName)
print(sc.master)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
img = Image.open("Spark_Master_Types.png")

# Show image
plt.figure(figsize=(12, 6)) 
plt.imshow(img)
plt.axis("off")
plt.show()

# 📂 Reading Data with Spark (From Your Notebooks)

In [ ]:
df = spark.read.csv(r"/home/bhuvaneshwaran/Desktop/Medium/csvFiles/Employee.csv", header=True, inferSchema=True)

### ✔ Explanation of each parameter:
🏷 header=True

Use the first row as column names
Without header → Spark names columns _c0, _c1, _c2...

📐 inferSchema=True

Spark automatically detects data types:

1. integers

2. strings

3. floats

4. timestamps

If False, all columns become string.

### ❗ Should you always use inferSchema?

No.

For production → define schema manually.

## ✔ Alternatives to CSV
* spark.read.json()

* spark.read.parquet()

* spark.read.orc()

* spark.read.text()

* spark.read.jdbc()

**Parquet** is recommended for performance.


## 🛑 Stopping Spark Session

In [ ]:
spark.stop()

Good practice because:

* Releases memory

* Stops JVM backend

* Avoids “multiple SparkContext” errors

🎯 Summary of What We Covered

* What SparkSession is

* Why it’s the entry point

* How .builder works

* Why appName, master, configs matter

* What SparkContext does

* How clusters are chosen

* Reading CSV with real examples

* Explanation of every parameter

* Best practices + alternatives

This is an excellent foundation.

In [ ]:
spark.stop()

## 🔥 Creating RDDs in PySpark
✔ Method 1: parallelize (convert Python list → RDD)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
data = [1, 2, 3, 4, 5]

In [ ]:
rdd = sc.parallelize([1,2,3,4,5])

When to use:

* For testing

* Small manual data

✔ Method 2: Load from external files

In [ ]:
rdd = sc.textFile(r"/home/bhuvaneshwaran/Desktop/Medium/txtFiles/simple.txt")

In [ ]:
print(rdd)

### When to use:

* Large dataset

* Streaming pipelines

* Log processing

## ⚡ Transformations vs Actions (Core Spark Concept)

RDD operations are split into two types:

## 🔵 Transformations:

These define the computation

**(MAP, FILTER, FLATMAP, REDUCEBYKEY)**

But they don’t run immediately.

Examples:

In [ ]:
rdd.map(lambda x: x*2)
rdd.filter(lambda x: x > 10)

These are lazy — Spark waits until it sees an action.

## 🔴 Actions:

These trigger the computation
They tell Spark: “Okay, now give me the result.”

In [ ]:
rdd.collect()

In [ ]:
rdd.take(5)

In [ ]:
rdd.count()

## 🧠 Understanding Lazy Evaluation (Beginner-Friendly Story)

Imagine writing steps:

* wash vegetables

* cut vegetables

* cook vegetables

But you start only when someone says:

**“Serve the food!”**

That's how **Spark** works.

You tell Spark the steps (transformations).
Spark waits.
When an action arrives → it runs the entire chain.

Result:

* Optimized execution

* Better performance

* Fewer unnecessary operations

## 🔥 Real Example: Transformations + Action

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])

transformed = rdd \
    .filter(lambda x: x % 2 == 0) \
    .map(lambda x: x * 10)

result = transformed.collect()

print(result)


### Explanation:

**filter** → keep even numbers

**map** → multiply by 10

**collect** → triggers execution

## 🌉 Narrow vs Wide Transformations

This is SUPER important for performance.

## 🔵 Narrow Transformation

Data stays on the same partition:

Examples:

* map

* filter

* flatMap

Fast because NO SHUFFLE.

## 🔴 Wide Transformation

Data needs to move across partitions → SHUFFLE

Examples:

* reduceByKey

* groupBy

* join

This is expensive and slow.

✔ Example:

In [ ]:
rdd.reduceByKey(lambda a,b: a+b)

This requires grouping keys → shuffle → slower.

## 💡 RDD Alternatives
✔ Modern alternative → DataFrames

Why better?

* Catalyst optimizer

* Columnar processing

* Faster execution

* Easier syntax

Example:

    df.filter(df.age > 30)


## ✔ Pandas API on Spark (pandas-on-spark)

Best for pandas users transitioning.

| Mistake                        | Why wrong             |
| ------------------------------ | --------------------- |
| Using `collect()` on huge data | Causes memory crash   |
| Too many wide transformations  | Slow jobs             |
| Creating many small partitions | Overhead              |
| Using RDD everywhere           | DataFrames are faster |
